# Value at Risk and Expected Shortfall for Portfolio

Calculation of Value at Risk and Expected Shortfall for a portfolio of stocks held by my family.

Portfolio consists of Indian stocks.

I have manually extracted the list of stocks from the broker generated csv report of current holdings as of 22nd December 2023.

Importing required libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt

initializing time window of 10 years

initializing stock data variables

In [3]:
end_date = dt.date.today()
start_date = end_date - dt.timedelta (3652)

stock_data = pd.read_csv(r"C:\Users\dhvanilzaveri\MBZPMSECTickers.csv")
stock_tickers = list(stock_data['Ticker'])
stock_qty = list(stock_data['Quantity'])

Defining a function to get stock data

In [10]:
def get_stockData(stocks, start, end):
    stockData = yf.download(stocks,start=start,end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    mean_Returns = returns.mean()
    cov_Matrix = returns.cov()
    return stockData,returns,mean_Returns,cov_Matrix


Function to measure portfolio performance

In [7]:
def pf_performance(weights, mean_returns):
    pf_returns = np.sum(mean_returns*weights)
    return pf_returns

Calling functions to get stock data for tickers and cleaning data

In [11]:
close_prices, returns, mean_returns, cov_matrix = get_stockData(stock_tickers,start_date, end_date)
close_prices = close_prices.dropna()
returns = returns.dropna()

[*********************100%***********************]  91 of 91 completed


Calculating portfolio metrics - valuation and returns.
Assuming fixed allocation over the whole period

In [12]:
close_prices['portfolio'] = close_prices.dot(stock_qty)
returns['portfolio'] = close_prices['portfolio'].pct_change()